# Active learning for bent pipe system

This system has 3 features - $d, \langle v \rangle, \theta$ and the label is pressure drop $\Delta P$. 

In [1]:
# Import required functions

from alcfd import *
import warnings
warnings.filterwarnings('ignore')

## Prepare train and test data

In [2]:
# Load data pool
input_file = 'bent_pipe_train.dat'
data = pd.read_csv(input_file, delimiter=' ')
#Isolate test data
test_data = data.loc[ (data['Elbow angle'] > 120) & (data['Pipe_D'] > 0.07)]
X_test = np.stack((test_data['Pipe_D'].astype('float'), 
                  test_data['Elbow angle'].astype('float'), 
                  test_data['Inlet_V'].astype('float')), axis=-1)
y_test = np.array(test_data['Del_P'].astype('float'))
# Rest is training data pool
train_pool = data.loc[~((data['Elbow angle'] > 120) & (data['Pipe_D'] > 0.07) )]

In [3]:
# Define features and labels
features = ['Pipe_D', 'angle', 'Inlet_V']
label = 'Del_P'

# This code prodices training sets for 5, 10,...,20 active learned points, with 5 runs for each
for i, j in enumerate(np.arange(5, 20, 5)):
    for k in range(5):
        X_train, y_train, X_pool, y_pool = sample_initial_training_points(train_pool, features, label)
        regressor_3d = active_learner(X_train, y_train)
        N_QUERIES = j
        Xs, ys, X_val, y_val = query_data(N_QUERIES, X_pool, y_pool, regressor_3d, 
                                          X_test=X_test, y_test=y_test)
        pred = regressor_3d.predict(X_test)
        directory = create_train_dat(f'cfd-sr-bent-pipe/active_learning_example/points{j}_{k}', Xs, ys, features, label)
print(f'Done!')

Done!
